---

stray thoughts: is goal-oriented word meaning what we need to be working towards? can RSA fully model this? can communicative contexts and past interactions be used to model word meaning?

In [ ]:
import numpy as np
from operator import itemgetter
import os, sys, re, random
from collections import defaultdict
from itertools import combinations

def row_norm(m):
    return np.divide(m.T, np.sum(m, axis=1)).T

def col_norm(m):
    return np.divide(m, np.sum(m, axis=0))

def safe_log(x):
    with np.errstate(divide='ignore'):
        return np.log(x)
    
def inner_product(x, y):
    return np.dot(x, y)

def powerset(x, minsize=0, maxsize=None):
    result = []
    if maxsize == None: maxsize = len(x)
    for i in range(minsize, maxsize+1):
        for val in combinations(x, i): result.append(list(val))
    return result

def mean_sq_error(x, y):
    return np.mean((x-y)**2)

def display_matrix(m, rnames=None, cnames=None, title='', digits=4):
    rwidth = 2 + max([len(x) for x in rnames] + [digits+2])
    cwidth = 2 + max([len(x) for x in cnames] + [digits+2])
    m = np.round(m, digits)
    s = ''; divider = ''; linebreak = '\n';
    for i in range(m.shape[0]):
        rowcontents = divider.join(str(x).rjust(cwidth) for x in m[i, :])
        s += str(rnames[i]).rjust(rwidth) + divider + rowcontents + linebreak
    print s
    
m = np.matrix([[1.0, 2.0], [3.0, 4.0]])
m = row_norm(m); print m
m = col_norm(m); print m

#### Define encapsulating class

class Module:
    def __init__(self,
                lexica=None,
                baselexicon=None,
                states=None,
                costs=None,
                messages=None,
                prior=None,
                lexprior=None,
                lexcount=None, 
                temperature=1.0,
                alpha=1.0,
                beta=1.0,
                nullmsg=True,
                nullcost=5.0):
        self.lexica = lexica
        self.baselexicon = baselexicon
        self.states = states
        self.costs = costs
        self.messages = messages
        self.prior = prior
        self.lexprior = lexprior
        self.lexcount = lexcount
        self.temperature = temperature
        self.alpha = alpha
        self.beta = beta 
        self.nullmsg = nullmsg
        self.nullcost = nullcost
        
        #intialise base prior arrays 
        if type(self.prior) == type(None):
            val = 1.0/len(self.states)
            self.prior = np.repeat(val, len(self.states))
        if type(self.lexprior) == type(None) and self.lexcount != None:
            val = 1.0/len(self.lexcount)
            self.prior = np.repeat(val, len(self.lexcount))
        else:
            self.lexprior = defaultdict(lambda: 1.0)
        if type(self.costs) == type(None):
            self.costs = np.zeros(len(self.messages))
            if self.nullmsg:
                self.costs[-1] = self.nullcost
        self.final_listener = np.zeros((len(self.messages), len(self.states)))
        self.final_speaker = None  


####  Interaction iterative functions

    def rsa(self, lex=None):
        if lex is None: lex = self.baselexicon
        literal = self.l0(lex)
        speaker = self.S(literal)
        listener = self.L(speaker)
        return [literal, speaker, listener]

    def run_base_model(self, lex, n=2, display=True, digits=4):
        return self.run(
                    n=n, 
                    display=display, 
                    digits=digits,
                    initial_listener = self.l0(lex),
                    start_level=0)

    def run(self,
       initial_listener,
       n=2,
       display=True,
       digits=4,
       start_level=0,
       ):
    #langs
        langs = [initial_listener]
        for i in range(1, (n-1)*2, 2):
            langs.append(self.S(langs[i-1]))
            langs.append(self.L(langs[i]))
        
        if len(langs) < 2:
            self.final_speaker = None
            self.final_listener = langs[-1]
        else:
            self.final_speaker, self.final_listener = langs[-2:]
        
        if display:
            self.display_iteration(langs, start_level=start_level, digits=digits)
        return langs
    

#### Agents

    def l0(self, lex):
        return row_norm(lex*self.prior)

    def L(self, speaker):
        return self.l0(speaker.T)

    def S(self, listener):
        return row_norm(np.exp(self.temperature * ((self.alpha*safe_log(listener.T)) - self.costs)))

    def s1(self, lex):
        return self.S(self.l0(lex))

    def l1(self, lex):
        return self.L(self.s1(lex))

    def lex_likelihood(self):
        p = np.array([np.sum(self.s1(lex), axis=0) * self.lexprior[i] for i, lex in enumerate(self.lexica)])
        return col_norm(p)

    def listener_lexical_marginalisation(self, listener):
        return np.sum(listener, axis=1)

    def speaker_lexical_marginalisation(self, speaker):
        return row_norm(np.sum(speaker, axis=0))




     

### RSA Fundamentals:

Elements: literal listener $L_0$, pragmatic speaker $S_1$, pragmatic listener $L_2$, cost function on dialogue (fixed utterances?) $\kappa$, inverse temperature parameter $\alpha$, prior probabilities of referents $P$, prior probabilities (truth functions?) of utterances referring to referents $L$, current referent $r$, current utterance $u$.

Literal Listener:

$l_o(r\,|\,u, L)\,\propto\,L(u\,|\,r)\,P(r)$



Pragmatic Speaker:


$s_1(u\,|\,r, L)\,\propto\,e\,^{\alpha\log{l_o(r\,|\,u, L)}\,-\,\kappa(u)}$


Pragmatic Listener:

$l_2(r\,|\,u, L)\,\propto\,s_1(u\,|\,t,\,L)\,P(r)$


(Note: this back and forth nature addresses Grice's conversational implicature theory but poses an obvious paradoxical problem i.e., it is always optimal to soft-maximise the current previous agent i.e., use strategy $S_{n-1}$ at whatever iteration you are at and use Bayes ruleto invert their decision procedure. I think most of the work in this setting so far sets recursion depth = 3)

Final equilibirium state currently is when only one agent is optimal wrt. the game, the other is $nearly$ optimal i.e., off by one iteration.

### Elements

Lexicon, states, costs, prior probabilities, temperature parameter, base speaker/listener, pragmatic (final?) speaker/listener, iterations, lexcount

### Agents

$l_0$: base listener that combines the prior probabilties with truth-conditional lexicon i.e., inner product of $lex$ and $self.prior$

$l_1$: general listener that takes input from speaker and transposes matrix

$s_1$: general speaker that takes input from listener and uses formula above i.e., weight/reason by the tempature parameter and incorporate costs

$l_{uncertainty}$: for $n$ lexicons, this is a general listener that takes input from speaker, then performs operations for all lexicons i.e., reasons over the speaker's marginal and composes them together

### Functions

$rsa(lexicon)$: has a base listener, pragmatic speaker, pragmatic listener; iterates back and forth and returns the agents

Define uncertainty etc., in detail

### Utility Funtions

transpose, row normalisation, column normalisation, inner product, powerset